In [0]:
print("Akshat enters into data world !!!")

Akshat enters into data world !!!


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
flight_df = spark.read.format("csv")\
        .option("header", "false")\
        .option("inferschema", "false")\
        .option("mode", "FAILFAST")\
        .load("/FileStore/tables/flights_data.csv")
flight_df.show(5)

+-----------------+-------------------+-----+
|              _c0|                _c1|  _c2|
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferschema", "false")
    .option("mode", "FAILFAST")
    .load("/FileStore/tables/flights_data.csv")
)
flight_df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [0]:
flight_df_chgschema = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferschema", "true")
    .option("mode", "FAILFAST")
    .load("/FileStore/tables/flights_data.csv")
)
flight_df_chgschema.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df_chgschema.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [0]:
from pyspark.sql.types import StructType,StructField,StringType, IntegerType

In [0]:
my_schema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", IntegerType(), True)
])

In [0]:
flight_df_chgschema = (
    spark.read.format("csv")
    .option("header", "false")
    .option("skipRows", 3)
    .option("inferschema", "true")
    .schema(my_schema)
    .option("mode", "PERMISSIVE")
    .load("/FileStore/tables/flights_data.csv")
)
flight_df_chgschema.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
+-----------------+-------------------+-----+
only showing top 5 rows



##BAD RECORDS HANDLING

In [0]:
 /FileStore/tables/employee.csv

In [0]:
employee_df = (
    spark.read.format("csv")
    .option("header", "TRUE")
    .option("inferschema", "true")
    .option("mode", "PERMISSIVE")
    .load("/FileStore/tables/employee.csv")
)
employee_df.show()

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|   India|
|  4|Prantosh| 17|200000|     Kolkata|   India|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+



In [0]:
employee_df = (
    spark.read.format("csv")
    .option("header", "TRUE")
    .option("inferschema", "true")
    .option("mode", "DROPMALFORMED")
    .load("/FileStore/tables/employee.csv")
)
employee_df.show()

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        null|nominee5|
+---+------+---+------+------------+--------+



In [0]:
employee_df = (
    spark.read.format("csv")
    .option("header", "TRUE")
    .option("inferschema", "true")
    .option("mode", "FAILFAST")
    .load("/FileStore/tables/employee.csv")
)
employee_df.show()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1146064879764472>:8
      1 employee_df = (
      2     spark.read.format("csv")
      3     .option("header", "TRUE")
   (...)
      6     .load("/FileStore/tables/employee.csv")
      7 )
----> 8 employee_df.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertic

CREATING SCHEMA MANUALLY TO DETECT BAD RECORDS

In [0]:
employee_schema = StructType([
    StructField("id", IntegerType(),True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", IntegerType(), True),
    StructField("address", StringType(), True),
    StructField("nominee", StringType(), True),
    StructField("_corrupt_record", StringType(), True)
])

In [0]:
employee_df = (
    spark.read.format("csv")
    .option("header", "TRUE")
    .option("inferschema", "true")
    .option("mode", "PERMISSIVE")
    .schema(employee_schema)
    .load("/FileStore/tables/employee.csv")
)
employee_df.show(truncate=False) #truncate=False will show full data of corrupted records
display(employee_df)

+---+--------+---+------+------------+--------+-------------------------------------------+
|id |name    |age|salary|address     |nominee |_corrupt_record                            |
+---+--------+---+------+------------+--------+-------------------------------------------+
|1  |Manish  |26 |75000 |bihar       |nominee1|null                                       |
|2  |Nikita  |23 |100000|uttarpradesh|nominee2|null                                       |
|3  |Pritam  |22 |150000|Bangalore   |India   |3,Pritam,22,150000,Bangalore,India,nominee3|
|4  |Prantosh|17 |200000|Kolkata     |India   |4,Prantosh,17,200000,Kolkata,India,nominee4|
|5  |Vikash  |31 |300000|null        |nominee5|null                                       |
+---+--------+---+------+------------+--------+-------------------------------------------+



id,name,age,salary,address,nominee,_corrupt_record
1,Manish,26,75000,bihar,nominee1,null
2,Nikita,23,100000,uttarpradesh,nominee2,null
3,Pritam,22,150000,Bangalore,India,"3,Pritam,22,150000,Bangalore,India,nominee3"
4,Prantosh,17,200000,Kolkata,India,"4,Prantosh,17,200000,Kolkata,India,nominee4"
5,Vikash,31,300000,null,nominee5,null


In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/employee.csv,employee.csv,230,1749319617000
dbfs:/FileStore/tables/flights_data.csv,flights_data.csv,7121,1749144702000


In [0]:
employee_df = (
    spark.read.format("csv")
    .option("header", "TRUE")
    .option("inferschema", "true")
    #.option("mode", "PERMISSIVE") THIS IS NOT NEEDED FOR STORING DATA
    .schema(employee_schema)
    .option("badRecordsPath","/FileStore/tables/bad_record")
    .load("/FileStore/tables/employee.csv")
)
employee_df.show(truncate=False)

+---+------+---+------+------------+--------+---------------+
|id |name  |age|salary|address     |nominee |_corrupt_record|
+---+------+---+------+------------+--------+---------------+
|1  |Manish|26 |75000 |bihar       |nominee1|null           |
|2  |Nikita|23 |100000|uttarpradesh|nominee2|null           |
|5  |Vikash|31 |300000|null        |nominee5|null           |
+---+------+---+------+------------+--------+---------------+



In [0]:
%fs ls /FileStore/tables/bad_record/20250607T202723/bad_records/part-00000-cb230469-e23d-418c-93c9-dfe23b2738ee

path,name,size,modificationTime
dbfs:/FileStore/tables/bad_record/20250607T202723/bad_records/part-00000-cb230469-e23d-418c-93c9-dfe23b2738ee,part-00000-cb230469-e23d-418c-93c9-dfe23b2738ee,484,1749328046000


In [0]:
bad_rec = spark.read.format("json").load("/FileStore/tables/bad_record/20250607T202723/bad_records/part-00000-cb230469-e23d-418c-93c9-dfe23b2738ee")
display(bad_rec)

path,reason,record
dbfs:/FileStore/tables/employee.csv,"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 3,Pritam,22,150000,Bangalore,India,nominee3","3,Pritam,22,150000,Bangalore,India,nominee3"
dbfs:/FileStore/tables/employee.csv,"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 4,Prantosh,17,200000,Kolkata,India,nominee4","4,Prantosh,17,200000,Kolkata,India,nominee4"
